In [1]:
from pyspark.sql.types import *

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("HousingPrices") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()



base = spark.read.csv("file:///home/maria/Documents/SparkSQLPractice/MITAnalyticsEdge/mvtWeek1.csv",inferSchema=True,header=True)
print(base.head())
print(base.printSchema())
print(base.columns)

Row(ID=8951354, Date='12/31/12 23:15', LocationDescription='STREET', Arrest=False, Domestic=False, Beat=623, District='6', CommunityArea='69', Year=2012, Latitude=41.75628399, Longitude=-87.62164472)
root
 |-- ID: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- LocationDescription: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: string (nullable = true)
 |-- CommunityArea: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)

None
['ID', 'Date', 'LocationDescription', 'Arrest', 'Domestic', 'Beat', 'District', 'CommunityArea', 'Year', 'Latitude', 'Longitude']


In [2]:
'''
ID: a unique identifier for each observation
Date: the date the crime occurred
LocationDescription: the location where the crime occurred
Arrest: whether or not an arrest was made for the crime (TRUE if an arrest was made, and FALSE if an arrest was not made)
Domestic: whether or not the crime was a domestic crime, meaning that it was committed against a family member (TRUE if it was domestic, and FALSE if it was not domestic)
Beat: the area, or "beat" in which the crime occurred. This is the smallest regional division defined by the Chicago police department.
District: the police district in which the crime occured. Each district is composed of many beats, and are defined by the Chicago Police Department.
CommunityArea: the community area in which the crime occurred. Since the 1920s, Chicago has been divided into what are called "community areas", of which there are now 77. The community areas were devised in an attempt to create socially homogeneous regions.
Year: the year in which the crime occurred.
Latitude: the latitude of the location at which the crime occurred.
Longitude: the longitude of the location at which the crime occurred.
'''

'\nID: a unique identifier for each observation\nDate: the date the crime occurred\nLocationDescription: the location where the crime occurred\nArrest: whether or not an arrest was made for the crime (TRUE if an arrest was made, and FALSE if an arrest was not made)\nDomestic: whether or not the crime was a domestic crime, meaning that it was committed against a family member (TRUE if it was domestic, and FALSE if it was not domestic)\nBeat: the area, or "beat" in which the crime occurred. This is the smallest regional division defined by the Chicago police department.\nDistrict: the police district in which the crime occured. Each district is composed of many beats, and are defined by the Chicago Police Department.\nCommunityArea: the community area in which the crime occurred. Since the 1920s, Chicago has been divided into what are called "community areas", of which there are now 77. The community areas were devised in an attempt to create socially homogeneous regions.\nYear: the year

In [3]:
base.createOrReplaceTempView("crimes")
#Number of rows in data
qur = spark.sql("SELECT COUNT(*) as row_n FROM crimes")
print(qur.show())

+------+
| row_n|
+------+
|191641|
+------+

None


In [4]:
#Number of columns in data
print(len(base.columns))

11


In [5]:
#Max ID
qur_max=spark.sql("SELECT MAX(ID) as max_id FROM crimes")
print(qur_max.show())

+-------+
| max_id|
+-------+
|9181151|
+-------+

None


In [6]:
#Min Beat
qur_min=spark.sql("SELECT MIN(Beat) as min_beat FROM crimes")
print(qur_min.show())

+--------+
|min_beat|
+--------+
|     111|
+--------+

None


In [7]:
#Nr of crimes with arrest
qur_arr=spark.sql("SELECT COUNT(*) as nr_arrests FROM crimes WHERE Arrest=TRUE")
print(qur_arr.show())

+----------+
|nr_arrests|
+----------+
|     15536|
+----------+

None


In [8]:
#How many observations have a LocationDescription value of ALLEY?
qur_all=spark.sql("SELECT COUNT(*) as nr_arrests FROM crimes WHERE LocationDescription='ALLEY'")
print(qur_all.show())

+----------+
|nr_arrests|
+----------+
|      2308|
+----------+

None


In [27]:
from pyspark.sql.functions import unix_timestamp, from_unixtime, date_format,to_date
from datetime import datetime

from datetime import datetime
from pyspark.sql.functions import col,udf

#qur_all=spark.sql("SELECT Date as date FROM crimes")
#qur_all.show()


print(base.select("Date").show(10, truncate = False))

myfunc =  udf(lambda x: datetime.strptime(x, '%m/%d/%y %I:%M:%S %p'), TimestampType())
df = base.withColumn('Date_time', myfunc(col('Date'))).drop("Date")

df.select(df["Date_time"]).show(5)

#https://datascience-enthusiast.com/Python/DataFramesVsRDDsVsSQLSpark-Part5.html
#http://www.dbaglobe.com/2019/03/python-date-time-string-format.html


+--------------+
|Date          |
+--------------+
|12/31/12 23:15|
|12/31/12 22:00|
|12/31/12 22:00|
|12/31/12 22:00|
|12/31/12 21:30|
|12/31/12 20:30|
|12/31/12 20:10|
|12/31/12 20:00|
|12/31/12 19:00|
|12/31/12 18:00|
+--------------+
only showing top 10 rows

None


Py4JJavaError: An error occurred while calling o185.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 31.0 failed 1 times, most recent failure: Lost task 0.0 in stage 31.0 (TID 52, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/maria/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/maria/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/maria/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 345, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/maria/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 141,